In [1]:
import gc
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

import plotly.express as px

from atel.data import BookCollection

In [3]:
# load data
book_col = BookCollection(data_file='./data/book_col_271120.pkl')

# Total number of books
print(f'Number of books: {book_col.num_books}')

Loaded from disk: ./data/book_col_271120.pkl
Number of books: 803


# Book level

In [4]:
for key, value in book_col[0].code_dict.items():
    print(f"\t{key:25s}\t{value}")

	Genre                    	['Selvportræt']
	Attitude                 	['Bedømmelse positiv']
	Stavning                 	['Semifonetisk']
	Perspektiv               	Eksplicit fortæller
	Tekstbånd                	
	Fremstillingsform        	['Beskrivende']
	Semantisk univers        	['Mennesker']
	Holistisk vurdering      	1
	Stemmer                  	['En stemme']


In [5]:
data = []

for i, book in enumerate(book_col):
    if book.code_dict is None:
        print(f'Book with index {i} is None')
        continue
    book.code_dict['book_id'] = i 
    data.append(book.code_dict)

Book with index 509 is None


In [6]:
book_df = pd.DataFrame(data)

In [7]:
book_df.head()

,Genre,Attitude,Stavning,Perspektiv,Tekstbånd,Fremstillingsform,Semantisk univers,Holistisk vurdering,Stemmer,book_id,Forbindere,Interjektioner,Metaforbrug,Noter
0,[Selvportræt],[Bedømmelse positiv],[Semifonetisk],Eksplicit fortæller,,[Beskrivende],[Mennesker],1,[En stemme],0,NaN,NaN,NaN,NaN
1,[Rim og remser],NaN,[Fonetisk],Implicit fortæller,[Semantisk],[Beskrivende],[Vilde dyr],1,[En stemme],1,NaN,NaN,NaN,NaN
2,,[Følelse 1. person],[Semifonetisk],Eksplicit fortæller,NaN,[Beskrivende],[Mig selv],0,[En stemme],2,[Tilføjelse],NaN,NaN,NaN
3,[Hverdagsbeskrivelse],NaN,"[Semifonetisk, Fonetisk]",Implicit fortæller,NaN,[Beskrivende],[Kæledyr],1,[En stemme],3,NaN,NaN,NaN,NaN
4,[Hverdagsbeskrivelse],[Følelse 3. person],[Konventionel skrivning],Implicit fortæller,NaN,[Beskrivende],[Familie],2,[En stemme],4,NaN,[Attitude],NaN,NaN


In [ ]:
list_cols = [
    'Genre', 'Attitude', 'Stavning', 'Perspektiv', 'Tekstbånd', 'Fremstillingsform', 
    'Semantisk univers', 'Stemmer', 'Forbindere', 'Interjektioner'
]

In [196]:
## Replace empty strings with NaN values
book_df = book_df.replace('', np.NaN)

## Replace all NaN with empty lists
for col in list_cols:
    book_df.loc[book_df[col].isnull(), col] = book_df.loc[book_df[col].isnull(), col].apply(lambda x: [])

## Interjektion - 3 values are not in list
book_df.loc[book_df['Interjektioner'] == 'Attitude', 'Interjektioner'] = book_df.loc[book_df['Interjektioner'] == 'Attitude', 'Interjektioner'].apply(lambda x: [x])

## "Vilde dyr" is there twice due to typo
book_df['Semantisk univers'] = book_df['Semantisk univers'].apply(lambda x: ['Vilde dyr' if i=='Vilde dyr\xa0' else i for i in x])

In [197]:
## Since data is in lists, we use MultiLabelBinarizer to one-hot-encode
mlb = MultiLabelBinarizer()

## One Hot Book
ohb_df = book_df.copy(deep=True)

for col in list_cols:
    ohb_df = ohb_df.join(pd.DataFrame(mlb.fit_transform(ohb_df.pop(col)),
                                      columns=[f'{col}_{x}' for x in mlb.classes_],
                                      index=ohb_df.index))

In [198]:
ohb_df.shape

(802, 91)

In [201]:
ohb_df.head()

,Holistisk vurdering,book_id,Metaforbrug,Noter,Genre_Anmeldelse,Genre_Billedtekster,Genre_Eventyr,Genre_Fagbog,Genre_Fantasifortælling,Genre_Gyserfortælling,...,Stemmer_Flere konkrete stemmer,Stemmer_Flere potentielle stemmer,Forbindere_Alternativ,Forbindere_Betingelse,Forbindere_Modsætning,Forbindere_Tid,Forbindere_Tilføjelse,Forbindere_Årsag,Interjektioner_Attitude,Interjektioner_Effekt
0,1,0,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,3,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,4,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Sentence level

In [232]:
text_array = []
empty_sen = 0

## Goal - create an array with shape (b, p, s), containing all sentences
for book in book_col:
    for page in book.pages:
        for sentence in page:
            if sentence.text == "":
                empty_sen += 1
                continue
            text_array.append([sentence.text])

print(f'Number of empty strings: {empty_sen}')

Number of empty strings: 2596


In [237]:
np.array(text_array, dtype='object').shape

(8834, 1)

In [238]:
for book in book_col:
    for page in book.pages:
        for sentence in page:
            if sentence.text == "":
                empty_sen += 1
                continue
            print(sentence.code_dict)
    break

{'V-tid': 'Nutid', 'A-kerne': ['Menneskeriget'], 'Forfelt': 'Subjekt', 'A-person': '1. person ental', 'A-opbygning': ['Pronomen'], 'P-beskriver': ['Bedømmelse'], 'P-opbygning': ['Adjektiv'], 'V-procestype': 'Relationel', 'Tekst-billede-relation': 'Overensstemmelse', 'V-form': 'Nutid', 'A-realisering': 'Subjekt', 'Specificering': 'Almindelig'}
